
*run on full dataset, 1 epoch* 

* Dynamic, lmfcc, batch_size = 256, [256,256,256,256],lr = 0.0001, act_fun = "relu": 59.27%, 0.0338
* Dynamic, lmfcc, batch_size = 256, [256,256,256,256],**lr = 0.0002**, act_fun = "relu": 62.59%, 0.0312
* Dynamic, lmfcc, batch_size = 256, **[256, 512, 512, 256]**, lr = 0.0001, act_fun = "relu": 61.05%, 0.0325
* Dynamic, lmfcc, **batch_size = 512**, [256, 512, 512, 256], **lr = 0.0002**, act_fun = "relu": 59.62%, 0.0336
* Dynamic, lmfcc, batch_size = 256, **[256, 256, 256, 256, 256]**, lr = 0.0001, act_fun = "relu": 59.01%, 0.0338
* Dynamic, **mspec**, batch_size = 256, [256,256,256,256],lr = 0.0001, act_fun = "relu": 55.39%, 0.00365
* Dynamic, lmfcc, **batch_size = 512**, [256, 256, 256, 256], lr = 0.0001, act_fun = "relu": 55.27%, 0.0369
* Dynamic, lmfcc, batch_size = 256, [256,256,256,256],lr = 0.0001, **act_fun = "tanh"**: 54.29%, 0.0377
* Dynamic, lmfcc, batch_size = 256, **[128, 128, 128, 128]**, lr = 0.0001, act_fun = "relu": 53.61%, 0.0381
* **Non-dynamic**, lmfcc, batch_size = 256, [256,256,256,256],lr = 0.0001, act_fun = "relu": 47.63%, 0.0421
* **Non-dynamic**, **mspec**, batch_size = 256, [256,256,256,256],lr = 0.0001, act_fun = "relu": 43.50%, 0.0450


**overall comparison**
* dynamic > non dynamic
* lmfcc > mspec
* more nodes in each layer > less nodes
* higher learning rate > lower learning rate
* higher learning rate > bigger batch size && higher learning rate > bigger batchsize
* more layers >= less layers
* relu > tanh

In [1]:
from pytorch_example import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from loguru import logger
from datetime import datetime
from pathlib import Path
import os
from tqdm import tqdm
import time
import json

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

In [3]:
# setup logging so that you can follow training using TensorBoard (see https://pytorch.org/docs/stable/tensorboard.html)
writer = SummaryWriter()

## model with dynamic mspec

* dynamic mspec
* active_func = relu
* batch_size = 256
* hidden_layer_list = [256, 256, 256, 256]
* initial_lr = 0.0001

In [5]:
# set parameters and initiate net
use_dynamic_features = True
feature_type = 'mspec'
activate_func = 'relu'
batch_size = 256
hidden_layer_list = [256, 256, 256, 256]
initial_lr = 0.0001
num_epochs = 1
if feature_type == 'lmfcc':
    feat_dim = 13
elif feature_type == 'mspec':
    feat_dim = 40

if use_dynamic_features:
    input_size = feat_dim * 7
else:
    input_size = feat_dim

stateList = np.load('state_list.npy').tolist()
num_cls = len(stateList)

net = Net(input_size=input_size, num_cls=num_cls, hidden_layer_list=hidden_layer_list, activate_func=activate_func)
logger.info(net)
logger.info('number of prameters:', count_parameters(net))
net = net.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=initial_lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

# load preprocessed data
prepared_data_train = np.load('prepared_train_data.npz')
if use_dynamic_features:
    np_train_x = prepared_data_train[f'data_x_dynamic_{feature_type}']
    np_train_y = prepared_data_train['data_y']
else:
    np_train_x = prepared_data_train[f'data_x_{feature_type}']
    np_train_y = prepared_data_train['data_y']
train_x = torch.tensor(np_train_x)
train_y = F.one_hot(torch.tensor(np_train_y, dtype=torch.long), num_classes=num_cls).float()

prepared_data_val = np.load('prepared_val_data.npz')
if use_dynamic_features:
    np_val_x = prepared_data_val[f'data_x_dynamic_{feature_type}']
    np_val_y = prepared_data_val['data_y']
else:
    np_val_x = prepared_data_val[f'data_x_{feature_type}']
    np_val_y = prepared_data_val['data_y']
val_x = torch.tensor(np_val_x)
val_y = F.one_hot(torch.tensor(np_val_y, dtype=torch.long), num_classes=num_cls).float()
    
prepared_data_test = np.load('prepared_test_data.npz')
if use_dynamic_features:
    np_test_x = prepared_data_test[f'data_x_dynamic_{feature_type}']
    np_test_y = prepared_data_test['data_y']
else:
    np_test_x = prepared_data_test[f'data_x_{feature_type}']
    np_test_y = prepared_data_test['data_y']
test_x = torch.tensor(np_test_x)
test_y = F.one_hot(torch.tensor(np_test_y, dtype=torch.long), num_classes=num_cls).float()

# create the data loaders for training and validation sets
train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(val_x, val_y)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# train the network
st = time.time()
last_st = time.time()
log_interval = 100

for epoch in range(num_epochs):
    net.train()
    train_loss = 0.0
    sub_train_loss = 0.0
    epoch_acc = 0.0
    cnt = 0
    for idx, (inputs, labels) in enumerate(train_loader):

        # move data from cpu to gpu (if available)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # accumulate the training loss
        train_loss += loss.item()

        sub_train_loss += loss.item()

        if idx % log_interval == 0:
            duration = time.time() - st
            acc = calc_accuracy(labels, outputs)
            epoch_acc += (acc * batch_size)
            cnt += batch_size
            logger.info(f'epoch {epoch:03d} / {num_epochs}, step {idx:04d} / {len(train_loader)}, loss {sub_train_loss / log_interval:.5f}, acc {acc*100:.2f}%, total time [{format_time(duration)}], one epoch time {format_time(len(train_loader) / log_interval * (time.time() - last_st))}')
            sub_train_loss = 0.0
            last_st = time.time()

    # update the learning rate
    scheduler.step()

    # calculate the validation loss
    net.eval()
    val_loss = 0.0
    val_epoch_acc = 0.0
    val_cnt = 0
    with torch.no_grad():
        val_loss = 0.0
        logger.info('Validation...')
        for inputs, labels in tqdm(val_loader):
            # move data from cpu to gpu (if available)
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            acc = calc_accuracy(labels, outputs)
            val_epoch_acc += (acc * batch_size)
            val_cnt += len(inputs)

    # print the epoch loss
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    train_acc = epoch_acc/cnt*100
    val_acc = val_epoch_acc/val_cnt*100

    logger.info(f'Epoch {epoch}: train_loss={train_loss:.4f}, train_acc={train_acc:.2f}%, val_loss={val_loss:.4f}, val_acc={val_acc:.2f}%, lr={scheduler.get_last_lr()[0]}')
    writer.add_scalars('loss',{'train':train_loss,'val':val_loss},epoch)
    
# finally evaluate model on the test set here
net.eval()
test_loss = 0.0
test_epoch_acc = 0.0
test_cnt = 0
logger.info('Testing...')
with torch.no_grad():
    test_loss = 0.0
    for inputs, labels in tqdm(test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        acc = calc_accuracy(labels, outputs)
        test_epoch_acc += (acc * batch_size)
        test_cnt += len(inputs)
test_loss /= len(test_loader)
test_acc = test_epoch_acc/test_cnt*100
logger.info(f'Results: test_loss={test_loss:.4f}, test_acc={test_acc:.2f}%')

2024-05-12 14:47:33.286 | INFO     | __main__:<module>:23 - Net(
  (activate): ReLU()
  (layer1): Linear(in_features=280, out_features=256, bias=True)
  (hidden_layers): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
  )
  (header_layer): Linear(in_features=256, out_features=61, bias=True)
)
2024-05-12 14:47:33.287 | INFO     | __main__:<module>:24 - number of prameters:
2024-05-12 14:47:37.307 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0000 / 5267, loss 0.00686, acc 0.39%, total time [00:00], one epoch time 00:06
2024-05-12 14:47:38.450 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0100 / 5267, loss 0.30431, acc 53.12%, total time [00:01], one epoch time 01:00
2024-05-12 14:47:39.520 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0200 / 5267, loss 0.06476

KeyboardInterrupt: 

## model with dynamic lmfcc

* dynamic lmfcc
* active_func = relu
* batch_size = 256
* hidden_layer_list = [256, 256, 256, 256]
* initial_lr = 0.0001

In [44]:
# set parameters and initiate net
use_dynamic_features = True
feature_type = 'lmfcc'
activate_func = 'relu'
batch_size = 256
hidden_layer_list = [256, 256, 256, 256]
initial_lr = 0.0001
num_epochs = 1
if feature_type == 'lmfcc':
    feat_dim = 13
elif feature_type == 'mspec':
    feat_dim = 40

if use_dynamic_features:
    input_size = feat_dim * 7
else:
    input_size = feat_dim

stateList = np.load('state_list.npy').tolist()
num_cls = len(stateList)

net = Net(input_size=input_size, num_cls=num_cls, hidden_layer_list=hidden_layer_list, activate_func=activate_func)
logger.info(net)
logger.info('number of prameters:', count_parameters(net))
net = net.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=initial_lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

# load preprocessed data
prepared_data_train = np.load('prepared_train_data.npz')
if use_dynamic_features:
    np_train_x = prepared_data_train[f'data_x_dynamic_{feature_type}']
    np_train_y = prepared_data_train['data_y']
else:
    np_train_x = prepared_data_train[f'data_x_{feature_type}']
    np_train_y = prepared_data_train['data_y']
train_x = torch.tensor(np_train_x)
train_y = F.one_hot(torch.tensor(np_train_y, dtype=torch.long), num_classes=num_cls).float()

prepared_data_val = np.load('prepared_val_data.npz')
if use_dynamic_features:
    np_val_x = prepared_data_val[f'data_x_dynamic_{feature_type}']
    np_val_y = prepared_data_val['data_y']
else:
    np_val_x = prepared_data_val[f'data_x_{feature_type}']
    np_val_y = prepared_data_val['data_y']
val_x = torch.tensor(np_val_x)
val_y = F.one_hot(torch.tensor(np_val_y, dtype=torch.long), num_classes=num_cls).float()
    
prepared_data_test = np.load('prepared_test_data.npz')
if use_dynamic_features:
    np_test_x = prepared_data_test[f'data_x_dynamic_{feature_type}']
    np_test_y = prepared_data_test['data_y']
else:
    np_test_x = prepared_data_test[f'data_x_{feature_type}']
    np_test_y = prepared_data_test['data_y']
test_x = torch.tensor(np_test_x)
test_y = F.one_hot(torch.tensor(np_test_y, dtype=torch.long), num_classes=num_cls).float()

# create the data loaders for training and validation sets
train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(val_x, val_y)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# train the network
st = time.time()
last_st = time.time()
log_interval = 100

for epoch in range(num_epochs):
    net.train()
    train_loss = 0.0
    sub_train_loss = 0.0
    epoch_acc = 0.0
    cnt = 0
    for idx, (inputs, labels) in enumerate(train_loader):

        # move data from cpu to gpu (if available)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # accumulate the training loss
        train_loss += loss.item()

        sub_train_loss += loss.item()

        if idx % log_interval == 0:
            duration = time.time() - st
            acc = calc_accuracy(labels, outputs)
            epoch_acc += (acc * batch_size)
            cnt += batch_size
            logger.info(f'epoch {epoch:03d} / {num_epochs}, step {idx:04d} / {len(train_loader)}, loss {sub_train_loss / log_interval:.5f}, acc {acc*100:.2f}%, total time [{format_time(duration)}], one epoch time {format_time(len(train_loader) / log_interval * (time.time() - last_st))}')
            sub_train_loss = 0.0
            last_st = time.time()

    # update the learning rate
    scheduler.step()

    # calculate the validation loss
    net.eval()
    val_loss = 0.0
    val_epoch_acc = 0.0
    val_cnt = 0
    with torch.no_grad():
        val_loss = 0.0
        logger.info('Validation...')
        for inputs, labels in tqdm(val_loader):
            # move data from cpu to gpu (if available)
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            acc = calc_accuracy(labels, outputs)
            val_epoch_acc += (acc * batch_size)
            val_cnt += len(inputs)

    # print the epoch loss
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    train_acc = epoch_acc/cnt*100
    val_acc = val_epoch_acc/val_cnt*100

    logger.info(f'Epoch {epoch}: train_loss={train_loss:.4f}, train_acc={train_acc:.2f}%, val_loss={val_loss:.4f}, val_acc={val_acc:.2f}%, lr={scheduler.get_last_lr()[0]}')
    writer.add_scalars('loss',{'train':train_loss,'val':val_loss},epoch)
    
# finally evaluate model on the test set here
net.eval()
test_loss = 0.0
test_epoch_acc = 0.0
test_cnt = 0
logger.info('Testing...')
with torch.no_grad():
    test_loss = 0.0
    for inputs, labels in tqdm(test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        acc = calc_accuracy(labels, outputs)
        test_epoch_acc += (acc * batch_size)
        test_cnt += len(inputs)
test_loss /= len(test_loader)
test_acc = test_epoch_acc/test_cnt*100
logger.info(f'Results: test_loss={test_loss:.4f}, test_acc={test_acc:.2f}%')

2024-05-10 10:12:56.914 | INFO     | __main__:<module>:23 - Net(
  (activate): ReLU()
  (layer1): Linear(in_features=91, out_features=256, bias=True)
  (hidden_layers): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
  )
  (header_layer): Linear(in_features=256, out_features=61, bias=True)
)
2024-05-10 10:12:56.916 | INFO     | __main__:<module>:24 - number of prameters:


2024-05-10 10:12:58.491 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0000 / 5267, loss 0.00694, acc 1.95%, total time [00:00], one epoch time 00:04
2024-05-10 10:12:59.022 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0100 / 5267, loss 0.39464, acc 13.67%, total time [00:00], one epoch time 00:27
2024-05-10 10:12:59.555 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0200 / 5267, loss 0.08115, acc 15.62%, total time [00:01], one epoch time 00:28
2024-05-10 10:13:00.100 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0300 / 5267, loss 0.07248, acc 21.09%, total time [00:01], one epoch time 00:28
2024-05-10 10:13:00.645 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0400 / 5267, loss 0.06695, acc 24.22%, total time [00:02], one epoch time 00:28
2024-05-10 10:13:01.189 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0500 / 5267, loss 0.06177, acc 29.30%, total time [00:02], one epoch time 00:28
2024-05-10 10:13:01.734 | INFO     

## non dynamic model with mspec

* non dynamic mspec
* active_func = relu
* batch_size = 256
* hidden_layer_list = [256, 256, 256, 256]
* initial_lr = 0.0001

In [45]:
# set parameters and initiate net
use_dynamic_features = False
feature_type = 'mspec'
activate_func = 'relu'
batch_size = 256
hidden_layer_list = [256, 256, 256, 256]
initial_lr = 0.0001
num_epochs = 1
if feature_type == 'lmfcc':
    feat_dim = 13
elif feature_type == 'mspec':
    feat_dim = 40

if use_dynamic_features:
    input_size = feat_dim * 7
else:
    input_size = feat_dim

stateList = np.load('state_list.npy').tolist()
num_cls = len(stateList)

net = Net(input_size=input_size, num_cls=num_cls, hidden_layer_list=hidden_layer_list, activate_func=activate_func)
logger.info(net)
logger.info('number of prameters:', count_parameters(net))
net = net.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=initial_lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

# load preprocessed data
prepared_data_train = np.load('prepared_train_data.npz')
if use_dynamic_features:
    np_train_x = prepared_data_train[f'data_x_dynamic_{feature_type}']
    np_train_y = prepared_data_train['data_y']
else:
    np_train_x = prepared_data_train[f'data_x_{feature_type}']
    np_train_y = prepared_data_train['data_y']
train_x = torch.tensor(np_train_x)
train_y = F.one_hot(torch.tensor(np_train_y, dtype=torch.long), num_classes=num_cls).float()

prepared_data_val = np.load('prepared_val_data.npz')
if use_dynamic_features:
    np_val_x = prepared_data_val[f'data_x_dynamic_{feature_type}']
    np_val_y = prepared_data_val['data_y']
else:
    np_val_x = prepared_data_val[f'data_x_{feature_type}']
    np_val_y = prepared_data_val['data_y']
val_x = torch.tensor(np_val_x)
val_y = F.one_hot(torch.tensor(np_val_y, dtype=torch.long), num_classes=num_cls).float()
    
prepared_data_test = np.load('prepared_test_data.npz')
if use_dynamic_features:
    np_test_x = prepared_data_test[f'data_x_dynamic_{feature_type}']
    np_test_y = prepared_data_test['data_y']
else:
    np_test_x = prepared_data_test[f'data_x_{feature_type}']
    np_test_y = prepared_data_test['data_y']
test_x = torch.tensor(np_test_x)
test_y = F.one_hot(torch.tensor(np_test_y, dtype=torch.long), num_classes=num_cls).float()

# create the data loaders for training and validation sets
train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(val_x, val_y)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# train the network
st = time.time()
last_st = time.time()
log_interval = 100

for epoch in range(num_epochs):
    net.train()
    train_loss = 0.0
    sub_train_loss = 0.0
    epoch_acc = 0.0
    cnt = 0
    for idx, (inputs, labels) in enumerate(train_loader):

        # move data from cpu to gpu (if available)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # accumulate the training loss
        train_loss += loss.item()

        sub_train_loss += loss.item()

        if idx % log_interval == 0:
            duration = time.time() - st
            acc = calc_accuracy(labels, outputs)
            epoch_acc += (acc * batch_size)
            cnt += batch_size
            logger.info(f'epoch {epoch:03d} / {num_epochs}, step {idx:04d} / {len(train_loader)}, loss {sub_train_loss / log_interval:.5f}, acc {acc*100:.2f}%, total time [{format_time(duration)}], one epoch time {format_time(len(train_loader) / log_interval * (time.time() - last_st))}')
            sub_train_loss = 0.0
            last_st = time.time()

    # update the learning rate
    scheduler.step()

    # calculate the validation loss
    net.eval()
    val_loss = 0.0
    val_epoch_acc = 0.0
    val_cnt = 0
    with torch.no_grad():
        val_loss = 0.0
        logger.info('Validation...')
        for inputs, labels in tqdm(val_loader):
            # move data from cpu to gpu (if available)
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            acc = calc_accuracy(labels, outputs)
            val_epoch_acc += (acc * batch_size)
            val_cnt += len(inputs)

    # print the epoch loss
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    train_acc = epoch_acc/cnt*100
    val_acc = val_epoch_acc/val_cnt*100

    logger.info(f'Epoch {epoch}: train_loss={train_loss:.4f}, train_acc={train_acc:.2f}%, val_loss={val_loss:.4f}, val_acc={val_acc:.2f}%, lr={scheduler.get_last_lr()[0]}')
    writer.add_scalars('loss',{'train':train_loss,'val':val_loss},epoch)
    
# finally evaluate model on the test set here
net.eval()
test_loss = 0.0
test_epoch_acc = 0.0
test_cnt = 0
logger.info('Testing...')
with torch.no_grad():
    test_loss = 0.0
    for inputs, labels in tqdm(test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        acc = calc_accuracy(labels, outputs)
        test_epoch_acc += (acc * batch_size)
        test_cnt += len(inputs)
test_loss /= len(test_loader)
test_acc = test_epoch_acc/test_cnt*100
logger.info(f'Results: test_loss={test_loss:.4f}, test_acc={test_acc:.2f}%')

2024-05-10 10:14:03.987 | INFO     | __main__:<module>:23 - Net(
  (activate): ReLU()
  (layer1): Linear(in_features=40, out_features=256, bias=True)
  (hidden_layers): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
  )
  (header_layer): Linear(in_features=256, out_features=61, bias=True)
)
2024-05-10 10:14:03.988 | INFO     | __main__:<module>:24 - number of prameters:
2024-05-10 10:14:04.837 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0000 / 5267, loss 0.00693, acc 0.39%, total time [00:00], one epoch time 00:04
2024-05-10 10:14:05.400 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0100 / 5267, loss 0.38057, acc 12.89%, total time [00:00], one epoch time 00:29
2024-05-10 10:14:05.955 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0200 / 5267, loss 0.09344,

## model with non dynamic lmfcc

* non dynamic lmfcc
* active_func = relu
* batch_size = 256
* hidden_layer_list = [256, 256, 256, 256]
* initial_lr = 0.0001

In [46]:
# set parameters and initiate net
use_dynamic_features = False
feature_type = 'lmfcc'
activate_func = 'relu'
batch_size = 256
hidden_layer_list = [256, 256, 256, 256]
initial_lr = 0.0001
num_epochs = 1
if feature_type == 'lmfcc':
    feat_dim = 13
elif feature_type == 'mspec':
    feat_dim = 40

if use_dynamic_features:
    input_size = feat_dim * 7
else:
    input_size = feat_dim

stateList = np.load('state_list.npy').tolist()
num_cls = len(stateList)

net = Net(input_size=input_size, num_cls=num_cls, hidden_layer_list=hidden_layer_list, activate_func=activate_func)
logger.info(net)
logger.info('number of prameters:', count_parameters(net))
net = net.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=initial_lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

# load preprocessed data
prepared_data_train = np.load('prepared_train_data.npz')
if use_dynamic_features:
    np_train_x = prepared_data_train[f'data_x_dynamic_{feature_type}']
    np_train_y = prepared_data_train['data_y']
else:
    np_train_x = prepared_data_train[f'data_x_{feature_type}']
    np_train_y = prepared_data_train['data_y']
train_x = torch.tensor(np_train_x)
train_y = F.one_hot(torch.tensor(np_train_y, dtype=torch.long), num_classes=num_cls).float()

prepared_data_val = np.load('prepared_val_data.npz')
if use_dynamic_features:
    np_val_x = prepared_data_val[f'data_x_dynamic_{feature_type}']
    np_val_y = prepared_data_val['data_y']
else:
    np_val_x = prepared_data_val[f'data_x_{feature_type}']
    np_val_y = prepared_data_val['data_y']
val_x = torch.tensor(np_val_x)
val_y = F.one_hot(torch.tensor(np_val_y, dtype=torch.long), num_classes=num_cls).float()
    
prepared_data_test = np.load('prepared_test_data.npz')
if use_dynamic_features:
    np_test_x = prepared_data_test[f'data_x_dynamic_{feature_type}']
    np_test_y = prepared_data_test['data_y']
else:
    np_test_x = prepared_data_test[f'data_x_{feature_type}']
    np_test_y = prepared_data_test['data_y']
test_x = torch.tensor(np_test_x)
test_y = F.one_hot(torch.tensor(np_test_y, dtype=torch.long), num_classes=num_cls).float()

# create the data loaders for training and validation sets
train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(val_x, val_y)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# train the network
st = time.time()
last_st = time.time()
log_interval = 100

for epoch in range(num_epochs):
    net.train()
    train_loss = 0.0
    sub_train_loss = 0.0
    epoch_acc = 0.0
    cnt = 0
    for idx, (inputs, labels) in enumerate(train_loader):

        # move data from cpu to gpu (if available)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # accumulate the training loss
        train_loss += loss.item()

        sub_train_loss += loss.item()

        if idx % log_interval == 0:
            duration = time.time() - st
            acc = calc_accuracy(labels, outputs)
            epoch_acc += (acc * batch_size)
            cnt += batch_size
            logger.info(f'epoch {epoch:03d} / {num_epochs}, step {idx:04d} / {len(train_loader)}, loss {sub_train_loss / log_interval:.5f}, acc {acc*100:.2f}%, total time [{format_time(duration)}], one epoch time {format_time(len(train_loader) / log_interval * (time.time() - last_st))}')
            sub_train_loss = 0.0
            last_st = time.time()

    # update the learning rate
    scheduler.step()

    # calculate the validation loss
    net.eval()
    val_loss = 0.0
    val_epoch_acc = 0.0
    val_cnt = 0
    with torch.no_grad():
        val_loss = 0.0
        logger.info('Validation...')
        for inputs, labels in tqdm(val_loader):
            # move data from cpu to gpu (if available)
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            acc = calc_accuracy(labels, outputs)
            val_epoch_acc += (acc * batch_size)
            val_cnt += len(inputs)

    # print the epoch loss
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    train_acc = epoch_acc/cnt*100
    val_acc = val_epoch_acc/val_cnt*100

    logger.info(f'Epoch {epoch}: train_loss={train_loss:.4f}, train_acc={train_acc:.2f}%, val_loss={val_loss:.4f}, val_acc={val_acc:.2f}%, lr={scheduler.get_last_lr()[0]}')
    writer.add_scalars('loss',{'train':train_loss,'val':val_loss},epoch)
    
# finally evaluate model on the test set here
net.eval()
test_loss = 0.0
test_epoch_acc = 0.0
test_cnt = 0
logger.info('Testing...')
with torch.no_grad():
    test_loss = 0.0
    for inputs, labels in tqdm(test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        acc = calc_accuracy(labels, outputs)
        test_epoch_acc += (acc * batch_size)
        test_cnt += len(inputs)
test_loss /= len(test_loader)
test_acc = test_epoch_acc/test_cnt*100
logger.info(f'Results: test_loss={test_loss:.4f}, test_acc={test_acc:.2f}%')

2024-05-10 10:15:19.674 | INFO     | __main__:<module>:23 - Net(
  (activate): ReLU()
  (layer1): Linear(in_features=13, out_features=256, bias=True)
  (hidden_layers): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
  )
  (header_layer): Linear(in_features=256, out_features=61, bias=True)
)
2024-05-10 10:15:19.676 | INFO     | __main__:<module>:24 - number of prameters:
2024-05-10 10:15:20.303 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0000 / 5267, loss 0.00695, acc 1.56%, total time [00:00], one epoch time 00:04
2024-05-10 10:15:20.896 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0100 / 5267, loss 0.40154, acc 10.55%, total time [00:00], one epoch time 00:31
2024-05-10 10:15:21.465 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0200 / 5267, loss 0.07818,

## model with dynamic lmfcc with different activate function

* dynamic lmfcc
* active_func = tanh
* batch_size = 256
* hidden_layer_list = [256, 256, 256, 256]
* initial_lr = 0.0001

In [47]:
# set parameters and initiate net
use_dynamic_features = True
feature_type = 'lmfcc'
activate_func = 'tanh'
batch_size = 256
hidden_layer_list = [256, 256, 256, 256]
initial_lr = 0.0001
num_epochs = 1
if feature_type == 'lmfcc':
    feat_dim = 13
elif feature_type == 'mspec':
    feat_dim = 40

if use_dynamic_features:
    input_size = feat_dim * 7
else:
    input_size = feat_dim

stateList = np.load('state_list.npy').tolist()
num_cls = len(stateList)

net = Net(input_size=input_size, num_cls=num_cls, hidden_layer_list=hidden_layer_list, activate_func=activate_func)
logger.info(net)
logger.info('number of prameters:', count_parameters(net))
net = net.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=initial_lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

# load preprocessed data
prepared_data_train = np.load('prepared_train_data.npz')
if use_dynamic_features:
    np_train_x = prepared_data_train[f'data_x_dynamic_{feature_type}']
    np_train_y = prepared_data_train['data_y']
else:
    np_train_x = prepared_data_train[f'data_x_{feature_type}']
    np_train_y = prepared_data_train['data_y']
train_x = torch.tensor(np_train_x)
train_y = F.one_hot(torch.tensor(np_train_y, dtype=torch.long), num_classes=num_cls).float()

prepared_data_val = np.load('prepared_val_data.npz')
if use_dynamic_features:
    np_val_x = prepared_data_val[f'data_x_dynamic_{feature_type}']
    np_val_y = prepared_data_val['data_y']
else:
    np_val_x = prepared_data_val[f'data_x_{feature_type}']
    np_val_y = prepared_data_val['data_y']
val_x = torch.tensor(np_val_x)
val_y = F.one_hot(torch.tensor(np_val_y, dtype=torch.long), num_classes=num_cls).float()
    
prepared_data_test = np.load('prepared_test_data.npz')
if use_dynamic_features:
    np_test_x = prepared_data_test[f'data_x_dynamic_{feature_type}']
    np_test_y = prepared_data_test['data_y']
else:
    np_test_x = prepared_data_test[f'data_x_{feature_type}']
    np_test_y = prepared_data_test['data_y']
test_x = torch.tensor(np_test_x)
test_y = F.one_hot(torch.tensor(np_test_y, dtype=torch.long), num_classes=num_cls).float()

# create the data loaders for training and validation sets
train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(val_x, val_y)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# train the network
st = time.time()
last_st = time.time()
log_interval = 100

for epoch in range(num_epochs):
    net.train()
    train_loss = 0.0
    sub_train_loss = 0.0
    epoch_acc = 0.0
    cnt = 0
    for idx, (inputs, labels) in enumerate(train_loader):

        # move data from cpu to gpu (if available)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # accumulate the training loss
        train_loss += loss.item()

        sub_train_loss += loss.item()

        if idx % log_interval == 0:
            duration = time.time() - st
            acc = calc_accuracy(labels, outputs)
            epoch_acc += (acc * batch_size)
            cnt += batch_size
            logger.info(f'epoch {epoch:03d} / {num_epochs}, step {idx:04d} / {len(train_loader)}, loss {sub_train_loss / log_interval:.5f}, acc {acc*100:.2f}%, total time [{format_time(duration)}], one epoch time {format_time(len(train_loader) / log_interval * (time.time() - last_st))}')
            sub_train_loss = 0.0
            last_st = time.time()

    # update the learning rate
    scheduler.step()

    # calculate the validation loss
    net.eval()
    val_loss = 0.0
    val_epoch_acc = 0.0
    val_cnt = 0
    with torch.no_grad():
        val_loss = 0.0
        logger.info('Validation...')
        for inputs, labels in tqdm(val_loader):
            # move data from cpu to gpu (if available)
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            acc = calc_accuracy(labels, outputs)
            val_epoch_acc += (acc * batch_size)
            val_cnt += len(inputs)

    # print the epoch loss
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    train_acc = epoch_acc/cnt*100
    val_acc = val_epoch_acc/val_cnt*100

    logger.info(f'Epoch {epoch}: train_loss={train_loss:.4f}, train_acc={train_acc:.2f}%, val_loss={val_loss:.4f}, val_acc={val_acc:.2f}%, lr={scheduler.get_last_lr()[0]}')
    writer.add_scalars('loss',{'train':train_loss,'val':val_loss},epoch)
    
# finally evaluate model on the test set here
net.eval()
test_loss = 0.0
test_epoch_acc = 0.0
test_cnt = 0
logger.info('Testing...')
with torch.no_grad():
    test_loss = 0.0
    for inputs, labels in tqdm(test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        acc = calc_accuracy(labels, outputs)
        test_epoch_acc += (acc * batch_size)
        test_cnt += len(inputs)
test_loss /= len(test_loader)
test_acc = test_epoch_acc/test_cnt*100
logger.info(f'Results: test_loss={test_loss:.4f}, test_acc={test_acc:.2f}%')

2024-05-10 10:18:24.399 | INFO     | __main__:<module>:23 - Net(
  (activate): Tanh()
  (layer1): Linear(in_features=91, out_features=256, bias=True)
  (hidden_layers): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): Tanh()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): Tanh()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): Tanh()
  )
  (header_layer): Linear(in_features=256, out_features=61, bias=True)
)
2024-05-10 10:18:24.400 | INFO     | __main__:<module>:24 - number of prameters:
2024-05-10 10:18:25.846 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0000 / 5267, loss 0.00693, acc 3.12%, total time [00:00], one epoch time 00:04
2024-05-10 10:18:26.412 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0100 / 5267, loss 0.41259, acc 1.17%, total time [00:00], one epoch time 00:29
2024-05-10 10:18:26.960 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0200 / 5267, loss 0.08312, 

## model with dynamic lmfcc with different layers

* dynamic lmfcc
* active_func = relu
* batch_size = 256
* hidden_layer_list = [256, 256, 256, 256, 256]
* initial_lr = 0.0001

In [48]:
# set parameters and initiate net
use_dynamic_features = True
feature_type = 'lmfcc'
activate_func = 'relu'
batch_size = 256
hidden_layer_list = [256, 256, 256, 256, 256]
initial_lr = 0.0001
num_epochs = 1
if feature_type == 'lmfcc':
    feat_dim = 13
elif feature_type == 'mspec':
    feat_dim = 40

if use_dynamic_features:
    input_size = feat_dim * 7
else:
    input_size = feat_dim

stateList = np.load('state_list.npy').tolist()
num_cls = len(stateList)

net = Net(input_size=input_size, num_cls=num_cls, hidden_layer_list=hidden_layer_list, activate_func=activate_func)
logger.info(net)
logger.info('number of prameters:', count_parameters(net))
net = net.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=initial_lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

# load preprocessed data
prepared_data_train = np.load('prepared_train_data.npz')
if use_dynamic_features:
    np_train_x = prepared_data_train[f'data_x_dynamic_{feature_type}']
    np_train_y = prepared_data_train['data_y']
else:
    np_train_x = prepared_data_train[f'data_x_{feature_type}']
    np_train_y = prepared_data_train['data_y']
train_x = torch.tensor(np_train_x)
train_y = F.one_hot(torch.tensor(np_train_y, dtype=torch.long), num_classes=num_cls).float()

prepared_data_val = np.load('prepared_val_data.npz')
if use_dynamic_features:
    np_val_x = prepared_data_val[f'data_x_dynamic_{feature_type}']
    np_val_y = prepared_data_val['data_y']
else:
    np_val_x = prepared_data_val[f'data_x_{feature_type}']
    np_val_y = prepared_data_val['data_y']
val_x = torch.tensor(np_val_x)
val_y = F.one_hot(torch.tensor(np_val_y, dtype=torch.long), num_classes=num_cls).float()
    
prepared_data_test = np.load('prepared_test_data.npz')
if use_dynamic_features:
    np_test_x = prepared_data_test[f'data_x_dynamic_{feature_type}']
    np_test_y = prepared_data_test['data_y']
else:
    np_test_x = prepared_data_test[f'data_x_{feature_type}']
    np_test_y = prepared_data_test['data_y']
test_x = torch.tensor(np_test_x)
test_y = F.one_hot(torch.tensor(np_test_y, dtype=torch.long), num_classes=num_cls).float()

# create the data loaders for training and validation sets
train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(val_x, val_y)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# train the network
st = time.time()
last_st = time.time()
log_interval = 100

for epoch in range(num_epochs):
    net.train()
    train_loss = 0.0
    sub_train_loss = 0.0
    epoch_acc = 0.0
    cnt = 0
    for idx, (inputs, labels) in enumerate(train_loader):

        # move data from cpu to gpu (if available)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # accumulate the training loss
        train_loss += loss.item()

        sub_train_loss += loss.item()

        if idx % log_interval == 0:
            duration = time.time() - st
            acc = calc_accuracy(labels, outputs)
            epoch_acc += (acc * batch_size)
            cnt += batch_size
            logger.info(f'epoch {epoch:03d} / {num_epochs}, step {idx:04d} / {len(train_loader)}, loss {sub_train_loss / log_interval:.5f}, acc {acc*100:.2f}%, total time [{format_time(duration)}], one epoch time {format_time(len(train_loader) / log_interval * (time.time() - last_st))}')
            sub_train_loss = 0.0
            last_st = time.time()

    # update the learning rate
    scheduler.step()

    # calculate the validation loss
    net.eval()
    val_loss = 0.0
    val_epoch_acc = 0.0
    val_cnt = 0
    with torch.no_grad():
        val_loss = 0.0
        logger.info('Validation...')
        for inputs, labels in tqdm(val_loader):
            # move data from cpu to gpu (if available)
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            acc = calc_accuracy(labels, outputs)
            val_epoch_acc += (acc * batch_size)
            val_cnt += len(inputs)

    # print the epoch loss
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    train_acc = epoch_acc/cnt*100
    val_acc = val_epoch_acc/val_cnt*100

    logger.info(f'Epoch {epoch}: train_loss={train_loss:.4f}, train_acc={train_acc:.2f}%, val_loss={val_loss:.4f}, val_acc={val_acc:.2f}%, lr={scheduler.get_last_lr()[0]}')
    writer.add_scalars('loss',{'train':train_loss,'val':val_loss},epoch)
    
# finally evaluate model on the test set here
net.eval()
test_loss = 0.0
test_epoch_acc = 0.0
test_cnt = 0
logger.info('Testing...')
with torch.no_grad():
    test_loss = 0.0
    for inputs, labels in tqdm(test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        acc = calc_accuracy(labels, outputs)
        test_epoch_acc += (acc * batch_size)
        test_cnt += len(inputs)
test_loss /= len(test_loader)
test_acc = test_epoch_acc/test_cnt*100
logger.info(f'Results: test_loss={test_loss:.4f}, test_acc={test_acc:.2f}%')

2024-05-10 10:19:46.929 | INFO     | __main__:<module>:23 - Net(
  (activate): ReLU()
  (layer1): Linear(in_features=91, out_features=256, bias=True)
  (hidden_layers): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): ReLU()
  )
  (header_layer): Linear(in_features=256, out_features=61, bias=True)
)
2024-05-10 10:19:46.931 | INFO     | __main__:<module>:24 - number of prameters:
2024-05-10 10:19:48.401 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0000 / 5267, loss 0.00693, acc 4.30%, total time [00:00], one epoch time 00:05
2024-05-10 10:19:49.061 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0100 / 5267, loss 0.38984, acc 12.50%, total time [00:00], one epoch time 00:34
2024-05-10 10:19:49.697 | INF

## model with dynamic lmfcc with different nodes

* dynamic lmfcc
* active_func = relu
* batch_size = 256
* hidden_layer_list = [256, 512, 512, 256]
* initial_lr = 0.0001

In [49]:
# set parameters and initiate net
use_dynamic_features = True
feature_type = 'lmfcc'
activate_func = 'relu'
batch_size = 256
hidden_layer_list = [256, 512, 512, 256]
initial_lr = 0.0001
num_epochs = 1
if feature_type == 'lmfcc':
    feat_dim = 13
elif feature_type == 'mspec':
    feat_dim = 40

if use_dynamic_features:
    input_size = feat_dim * 7
else:
    input_size = feat_dim

stateList = np.load('state_list.npy').tolist()
num_cls = len(stateList)

net = Net(input_size=input_size, num_cls=num_cls, hidden_layer_list=hidden_layer_list, activate_func=activate_func)
logger.info(net)
logger.info('number of prameters:', count_parameters(net))
net = net.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=initial_lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

# load preprocessed data
prepared_data_train = np.load('prepared_train_data.npz')
if use_dynamic_features:
    np_train_x = prepared_data_train[f'data_x_dynamic_{feature_type}']
    np_train_y = prepared_data_train['data_y']
else:
    np_train_x = prepared_data_train[f'data_x_{feature_type}']
    np_train_y = prepared_data_train['data_y']
train_x = torch.tensor(np_train_x)
train_y = F.one_hot(torch.tensor(np_train_y, dtype=torch.long), num_classes=num_cls).float()

prepared_data_val = np.load('prepared_val_data.npz')
if use_dynamic_features:
    np_val_x = prepared_data_val[f'data_x_dynamic_{feature_type}']
    np_val_y = prepared_data_val['data_y']
else:
    np_val_x = prepared_data_val[f'data_x_{feature_type}']
    np_val_y = prepared_data_val['data_y']
val_x = torch.tensor(np_val_x)
val_y = F.one_hot(torch.tensor(np_val_y, dtype=torch.long), num_classes=num_cls).float()
    
prepared_data_test = np.load('prepared_test_data.npz')
if use_dynamic_features:
    np_test_x = prepared_data_test[f'data_x_dynamic_{feature_type}']
    np_test_y = prepared_data_test['data_y']
else:
    np_test_x = prepared_data_test[f'data_x_{feature_type}']
    np_test_y = prepared_data_test['data_y']
test_x = torch.tensor(np_test_x)
test_y = F.one_hot(torch.tensor(np_test_y, dtype=torch.long), num_classes=num_cls).float()

# create the data loaders for training and validation sets
train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(val_x, val_y)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# train the network
st = time.time()
last_st = time.time()
log_interval = 100

for epoch in range(num_epochs):
    net.train()
    train_loss = 0.0
    sub_train_loss = 0.0
    epoch_acc = 0.0
    cnt = 0
    for idx, (inputs, labels) in enumerate(train_loader):

        # move data from cpu to gpu (if available)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # accumulate the training loss
        train_loss += loss.item()

        sub_train_loss += loss.item()

        if idx % log_interval == 0:
            duration = time.time() - st
            acc = calc_accuracy(labels, outputs)
            epoch_acc += (acc * batch_size)
            cnt += batch_size
            logger.info(f'epoch {epoch:03d} / {num_epochs}, step {idx:04d} / {len(train_loader)}, loss {sub_train_loss / log_interval:.5f}, acc {acc*100:.2f}%, total time [{format_time(duration)}], one epoch time {format_time(len(train_loader) / log_interval * (time.time() - last_st))}')
            sub_train_loss = 0.0
            last_st = time.time()

    # update the learning rate
    scheduler.step()

    # calculate the validation loss
    net.eval()
    val_loss = 0.0
    val_epoch_acc = 0.0
    val_cnt = 0
    with torch.no_grad():
        val_loss = 0.0
        logger.info('Validation...')
        for inputs, labels in tqdm(val_loader):
            # move data from cpu to gpu (if available)
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            acc = calc_accuracy(labels, outputs)
            val_epoch_acc += (acc * batch_size)
            val_cnt += len(inputs)

    # print the epoch loss
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    train_acc = epoch_acc/cnt*100
    val_acc = val_epoch_acc/val_cnt*100

    logger.info(f'Epoch {epoch}: train_loss={train_loss:.4f}, train_acc={train_acc:.2f}%, val_loss={val_loss:.4f}, val_acc={val_acc:.2f}%, lr={scheduler.get_last_lr()[0]}')
    writer.add_scalars('loss',{'train':train_loss,'val':val_loss},epoch)
    
# finally evaluate model on the test set here
net.eval()
test_loss = 0.0
test_epoch_acc = 0.0
test_cnt = 0
logger.info('Testing...')
with torch.no_grad():
    test_loss = 0.0
    for inputs, labels in tqdm(test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        acc = calc_accuracy(labels, outputs)
        test_epoch_acc += (acc * batch_size)
        test_cnt += len(inputs)
test_loss /= len(test_loader)
test_acc = test_epoch_acc/test_cnt*100
logger.info(f'Results: test_loss={test_loss:.4f}, test_acc={test_acc:.2f}%')

2024-05-10 10:24:56.229 | INFO     | __main__:<module>:23 - Net(
  (activate): ReLU()
  (layer1): Linear(in_features=91, out_features=256, bias=True)
  (hidden_layers): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
  )
  (header_layer): Linear(in_features=256, out_features=61, bias=True)
)
2024-05-10 10:24:56.230 | INFO     | __main__:<module>:24 - number of prameters:
2024-05-10 10:24:57.720 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0000 / 5267, loss 0.00688, acc 1.17%, total time [00:00], one epoch time 00:05
2024-05-10 10:24:58.305 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0100 / 5267, loss 0.29893, acc 14.06%, total time [00:00], one epoch time 00:30
2024-05-10 10:24:58.879 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0200 / 5267, loss 0.07622,

## model with dynamic lmfcc with different nodes

* dynamic lmfcc
* active_func = relu
* batch_size = 256
* hidden_layer_list = [128, 128, 128, 128]
* initial_lr = 0.0001

In [50]:
# set parameters and initiate net
use_dynamic_features = True
feature_type = 'lmfcc'
activate_func = 'relu'
batch_size = 256
hidden_layer_list = [128, 128, 128, 128]
initial_lr = 0.0001
num_epochs = 1
if feature_type == 'lmfcc':
    feat_dim = 13
elif feature_type == 'mspec':
    feat_dim = 40

if use_dynamic_features:
    input_size = feat_dim * 7
else:
    input_size = feat_dim

stateList = np.load('state_list.npy').tolist()
num_cls = len(stateList)

net = Net(input_size=input_size, num_cls=num_cls, hidden_layer_list=hidden_layer_list, activate_func=activate_func)
logger.info(net)
logger.info('number of prameters:', count_parameters(net))
net = net.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=initial_lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

# load preprocessed data
prepared_data_train = np.load('prepared_train_data.npz')
if use_dynamic_features:
    np_train_x = prepared_data_train[f'data_x_dynamic_{feature_type}']
    np_train_y = prepared_data_train['data_y']
else:
    np_train_x = prepared_data_train[f'data_x_{feature_type}']
    np_train_y = prepared_data_train['data_y']
train_x = torch.tensor(np_train_x)
train_y = F.one_hot(torch.tensor(np_train_y, dtype=torch.long), num_classes=num_cls).float()

prepared_data_val = np.load('prepared_val_data.npz')
if use_dynamic_features:
    np_val_x = prepared_data_val[f'data_x_dynamic_{feature_type}']
    np_val_y = prepared_data_val['data_y']
else:
    np_val_x = prepared_data_val[f'data_x_{feature_type}']
    np_val_y = prepared_data_val['data_y']
val_x = torch.tensor(np_val_x)
val_y = F.one_hot(torch.tensor(np_val_y, dtype=torch.long), num_classes=num_cls).float()
    
prepared_data_test = np.load('prepared_test_data.npz')
if use_dynamic_features:
    np_test_x = prepared_data_test[f'data_x_dynamic_{feature_type}']
    np_test_y = prepared_data_test['data_y']
else:
    np_test_x = prepared_data_test[f'data_x_{feature_type}']
    np_test_y = prepared_data_test['data_y']
test_x = torch.tensor(np_test_x)
test_y = F.one_hot(torch.tensor(np_test_y, dtype=torch.long), num_classes=num_cls).float()

# create the data loaders for training and validation sets
train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(val_x, val_y)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# train the network
st = time.time()
last_st = time.time()
log_interval = 100

for epoch in range(num_epochs):
    net.train()
    train_loss = 0.0
    sub_train_loss = 0.0
    epoch_acc = 0.0
    cnt = 0
    for idx, (inputs, labels) in enumerate(train_loader):

        # move data from cpu to gpu (if available)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # accumulate the training loss
        train_loss += loss.item()

        sub_train_loss += loss.item()

        if idx % log_interval == 0:
            duration = time.time() - st
            acc = calc_accuracy(labels, outputs)
            epoch_acc += (acc * batch_size)
            cnt += batch_size
            logger.info(f'epoch {epoch:03d} / {num_epochs}, step {idx:04d} / {len(train_loader)}, loss {sub_train_loss / log_interval:.5f}, acc {acc*100:.2f}%, total time [{format_time(duration)}], one epoch time {format_time(len(train_loader) / log_interval * (time.time() - last_st))}')
            sub_train_loss = 0.0
            last_st = time.time()

    # update the learning rate
    scheduler.step()

    # calculate the validation loss
    net.eval()
    val_loss = 0.0
    val_epoch_acc = 0.0
    val_cnt = 0
    with torch.no_grad():
        val_loss = 0.0
        logger.info('Validation...')
        for inputs, labels in tqdm(val_loader):
            # move data from cpu to gpu (if available)
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            acc = calc_accuracy(labels, outputs)
            val_epoch_acc += (acc * batch_size)
            val_cnt += len(inputs)

    # print the epoch loss
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    train_acc = epoch_acc/cnt*100
    val_acc = val_epoch_acc/val_cnt*100

    logger.info(f'Epoch {epoch}: train_loss={train_loss:.4f}, train_acc={train_acc:.2f}%, val_loss={val_loss:.4f}, val_acc={val_acc:.2f}%, lr={scheduler.get_last_lr()[0]}')
    writer.add_scalars('loss',{'train':train_loss,'val':val_loss},epoch)
    
# finally evaluate model on the test set here
net.eval()
test_loss = 0.0
test_epoch_acc = 0.0
test_cnt = 0
logger.info('Testing...')
with torch.no_grad():
    test_loss = 0.0
    for inputs, labels in tqdm(test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        acc = calc_accuracy(labels, outputs)
        test_epoch_acc += (acc * batch_size)
        test_cnt += len(inputs)
test_loss /= len(test_loader)
test_acc = test_epoch_acc/test_cnt*100
logger.info(f'Results: test_loss={test_loss:.4f}, test_acc={test_acc:.2f}%')

2024-05-10 10:26:33.284 | INFO     | __main__:<module>:23 - Net(
  (activate): ReLU()
  (layer1): Linear(in_features=91, out_features=128, bias=True)
  (hidden_layers): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): ReLU()
  )
  (header_layer): Linear(in_features=128, out_features=61, bias=True)
)
2024-05-10 10:26:33.285 | INFO     | __main__:<module>:24 - number of prameters:
2024-05-10 10:26:34.797 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0000 / 5267, loss 0.00691, acc 0.39%, total time [00:00], one epoch time 00:05
2024-05-10 10:26:35.300 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0100 / 5267, loss 0.59504, acc 1.17%, total time [00:00], one epoch time 00:26
2024-05-10 10:26:35.835 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0200 / 5267, loss 0.12221, 

## model with dynamic lmfcc with different batch size

* dynamic lmfcc
* active_func = relu
* batch_size = 512
* hidden_layer_list = [256, 256, 256, 256]
* initial_lr = 0.0001

In [51]:
# set parameters and initiate net
use_dynamic_features = True
feature_type = 'lmfcc'
activate_func = 'relu'
batch_size = 512
hidden_layer_list = [256, 256, 256, 256]
initial_lr = 0.0001
num_epochs = 1
if feature_type == 'lmfcc':
    feat_dim = 13
elif feature_type == 'mspec':
    feat_dim = 40

if use_dynamic_features:
    input_size = feat_dim * 7
else:
    input_size = feat_dim

stateList = np.load('state_list.npy').tolist()
num_cls = len(stateList)

net = Net(input_size=input_size, num_cls=num_cls, hidden_layer_list=hidden_layer_list, activate_func=activate_func)
logger.info(net)
logger.info('number of prameters:', count_parameters(net))
net = net.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=initial_lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

# load preprocessed data
prepared_data_train = np.load('prepared_train_data.npz')
if use_dynamic_features:
    np_train_x = prepared_data_train[f'data_x_dynamic_{feature_type}']
    np_train_y = prepared_data_train['data_y']
else:
    np_train_x = prepared_data_train[f'data_x_{feature_type}']
    np_train_y = prepared_data_train['data_y']
train_x = torch.tensor(np_train_x)
train_y = F.one_hot(torch.tensor(np_train_y, dtype=torch.long), num_classes=num_cls).float()

prepared_data_val = np.load('prepared_val_data.npz')
if use_dynamic_features:
    np_val_x = prepared_data_val[f'data_x_dynamic_{feature_type}']
    np_val_y = prepared_data_val['data_y']
else:
    np_val_x = prepared_data_val[f'data_x_{feature_type}']
    np_val_y = prepared_data_val['data_y']
val_x = torch.tensor(np_val_x)
val_y = F.one_hot(torch.tensor(np_val_y, dtype=torch.long), num_classes=num_cls).float()
    
prepared_data_test = np.load('prepared_test_data.npz')
if use_dynamic_features:
    np_test_x = prepared_data_test[f'data_x_dynamic_{feature_type}']
    np_test_y = prepared_data_test['data_y']
else:
    np_test_x = prepared_data_test[f'data_x_{feature_type}']
    np_test_y = prepared_data_test['data_y']
test_x = torch.tensor(np_test_x)
test_y = F.one_hot(torch.tensor(np_test_y, dtype=torch.long), num_classes=num_cls).float()

# create the data loaders for training and validation sets
train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(val_x, val_y)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# train the network
st = time.time()
last_st = time.time()
log_interval = 100

for epoch in range(num_epochs):
    net.train()
    train_loss = 0.0
    sub_train_loss = 0.0
    epoch_acc = 0.0
    cnt = 0
    for idx, (inputs, labels) in enumerate(train_loader):

        # move data from cpu to gpu (if available)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # accumulate the training loss
        train_loss += loss.item()

        sub_train_loss += loss.item()

        if idx % log_interval == 0:
            duration = time.time() - st
            acc = calc_accuracy(labels, outputs)
            epoch_acc += (acc * batch_size)
            cnt += batch_size
            logger.info(f'epoch {epoch:03d} / {num_epochs}, step {idx:04d} / {len(train_loader)}, loss {sub_train_loss / log_interval:.5f}, acc {acc*100:.2f}%, total time [{format_time(duration)}], one epoch time {format_time(len(train_loader) / log_interval * (time.time() - last_st))}')
            sub_train_loss = 0.0
            last_st = time.time()

    # update the learning rate
    scheduler.step()

    # calculate the validation loss
    net.eval()
    val_loss = 0.0
    val_epoch_acc = 0.0
    val_cnt = 0
    with torch.no_grad():
        val_loss = 0.0
        logger.info('Validation...')
        for inputs, labels in tqdm(val_loader):
            # move data from cpu to gpu (if available)
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            acc = calc_accuracy(labels, outputs)
            val_epoch_acc += (acc * batch_size)
            val_cnt += len(inputs)

    # print the epoch loss
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    train_acc = epoch_acc/cnt*100
    val_acc = val_epoch_acc/val_cnt*100

    logger.info(f'Epoch {epoch}: train_loss={train_loss:.4f}, train_acc={train_acc:.2f}%, val_loss={val_loss:.4f}, val_acc={val_acc:.2f}%, lr={scheduler.get_last_lr()[0]}')
    writer.add_scalars('loss',{'train':train_loss,'val':val_loss},epoch)
    
# finally evaluate model on the test set here
net.eval()
test_loss = 0.0
test_epoch_acc = 0.0
test_cnt = 0
logger.info('Testing...')
with torch.no_grad():
    test_loss = 0.0
    for inputs, labels in tqdm(test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        acc = calc_accuracy(labels, outputs)
        test_epoch_acc += (acc * batch_size)
        test_cnt += len(inputs)
test_loss /= len(test_loader)
test_acc = test_epoch_acc/test_cnt*100
logger.info(f'Results: test_loss={test_loss:.4f}, test_acc={test_acc:.2f}%')

2024-05-10 10:28:04.586 | INFO     | __main__:<module>:23 - Net(
  (activate): ReLU()
  (layer1): Linear(in_features=91, out_features=256, bias=True)
  (hidden_layers): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
  )
  (header_layer): Linear(in_features=256, out_features=61, bias=True)
)
2024-05-10 10:28:04.588 | INFO     | __main__:<module>:24 - number of prameters:
2024-05-10 10:28:06.010 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0000 / 2634, loss 0.00693, acc 1.56%, total time [00:00], one epoch time 00:02
2024-05-10 10:28:06.754 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0100 / 2634, loss 0.39057, acc 12.89%, total time [00:00], one epoch time 00:19
2024-05-10 10:28:07.408 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0200 / 2634, loss 0.08045,

## model with dynamic lmfcc with different batch size and different learning rate

* dynamic lmfcc
* active_func = relu
* batch_size = 512
* hidden_layer_list = [256, 256, 256, 256]
* initial_lr = 0.0002

In [52]:
# set parameters and initiate net
use_dynamic_features = True
feature_type = 'lmfcc'
activate_func = 'relu'
batch_size = 512
hidden_layer_list = [256, 256, 256, 256]
initial_lr = 0.0002
num_epochs = 1
if feature_type == 'lmfcc':
    feat_dim = 13
elif feature_type == 'mspec':
    feat_dim = 40

if use_dynamic_features:
    input_size = feat_dim * 7
else:
    input_size = feat_dim

stateList = np.load('state_list.npy').tolist()
num_cls = len(stateList)

net = Net(input_size=input_size, num_cls=num_cls, hidden_layer_list=hidden_layer_list, activate_func=activate_func)
logger.info(net)
logger.info('number of prameters:', count_parameters(net))
net = net.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=initial_lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

# load preprocessed data
prepared_data_train = np.load('prepared_train_data.npz')
if use_dynamic_features:
    np_train_x = prepared_data_train[f'data_x_dynamic_{feature_type}']
    np_train_y = prepared_data_train['data_y']
else:
    np_train_x = prepared_data_train[f'data_x_{feature_type}']
    np_train_y = prepared_data_train['data_y']
train_x = torch.tensor(np_train_x)
train_y = F.one_hot(torch.tensor(np_train_y, dtype=torch.long), num_classes=num_cls).float()

prepared_data_val = np.load('prepared_val_data.npz')
if use_dynamic_features:
    np_val_x = prepared_data_val[f'data_x_dynamic_{feature_type}']
    np_val_y = prepared_data_val['data_y']
else:
    np_val_x = prepared_data_val[f'data_x_{feature_type}']
    np_val_y = prepared_data_val['data_y']
val_x = torch.tensor(np_val_x)
val_y = F.one_hot(torch.tensor(np_val_y, dtype=torch.long), num_classes=num_cls).float()
    
prepared_data_test = np.load('prepared_test_data.npz')
if use_dynamic_features:
    np_test_x = prepared_data_test[f'data_x_dynamic_{feature_type}']
    np_test_y = prepared_data_test['data_y']
else:
    np_test_x = prepared_data_test[f'data_x_{feature_type}']
    np_test_y = prepared_data_test['data_y']
test_x = torch.tensor(np_test_x)
test_y = F.one_hot(torch.tensor(np_test_y, dtype=torch.long), num_classes=num_cls).float()

# create the data loaders for training and validation sets
train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(val_x, val_y)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# train the network
st = time.time()
last_st = time.time()
log_interval = 100

for epoch in range(num_epochs):
    net.train()
    train_loss = 0.0
    sub_train_loss = 0.0
    epoch_acc = 0.0
    cnt = 0
    for idx, (inputs, labels) in enumerate(train_loader):

        # move data from cpu to gpu (if available)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # accumulate the training loss
        train_loss += loss.item()

        sub_train_loss += loss.item()

        if idx % log_interval == 0:
            duration = time.time() - st
            acc = calc_accuracy(labels, outputs)
            epoch_acc += (acc * batch_size)
            cnt += batch_size
            logger.info(f'epoch {epoch:03d} / {num_epochs}, step {idx:04d} / {len(train_loader)}, loss {sub_train_loss / log_interval:.5f}, acc {acc*100:.2f}%, total time [{format_time(duration)}], one epoch time {format_time(len(train_loader) / log_interval * (time.time() - last_st))}')
            sub_train_loss = 0.0
            last_st = time.time()

    # update the learning rate
    scheduler.step()

    # calculate the validation loss
    net.eval()
    val_loss = 0.0
    val_epoch_acc = 0.0
    val_cnt = 0
    with torch.no_grad():
        val_loss = 0.0
        logger.info('Validation...')
        for inputs, labels in tqdm(val_loader):
            # move data from cpu to gpu (if available)
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            acc = calc_accuracy(labels, outputs)
            val_epoch_acc += (acc * batch_size)
            val_cnt += len(inputs)

    # print the epoch loss
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    train_acc = epoch_acc/cnt*100
    val_acc = val_epoch_acc/val_cnt*100

    logger.info(f'Epoch {epoch}: train_loss={train_loss:.4f}, train_acc={train_acc:.2f}%, val_loss={val_loss:.4f}, val_acc={val_acc:.2f}%, lr={scheduler.get_last_lr()[0]}')
    writer.add_scalars('loss',{'train':train_loss,'val':val_loss},epoch)
    
# finally evaluate model on the test set here
net.eval()
test_loss = 0.0
test_epoch_acc = 0.0
test_cnt = 0
logger.info('Testing...')
with torch.no_grad():
    test_loss = 0.0
    for inputs, labels in tqdm(test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        acc = calc_accuracy(labels, outputs)
        test_epoch_acc += (acc * batch_size)
        test_cnt += len(inputs)
test_loss /= len(test_loader)
test_acc = test_epoch_acc/test_cnt*100
logger.info(f'Results: test_loss={test_loss:.4f}, test_acc={test_acc:.2f}%')

2024-05-10 10:29:03.184 | INFO     | __main__:<module>:23 - Net(
  (activate): ReLU()
  (layer1): Linear(in_features=91, out_features=256, bias=True)
  (hidden_layers): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
  )
  (header_layer): Linear(in_features=256, out_features=61, bias=True)
)
2024-05-10 10:29:03.186 | INFO     | __main__:<module>:24 - number of prameters:
2024-05-10 10:29:04.610 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0000 / 2634, loss 0.00695, acc 0.59%, total time [00:00], one epoch time 00:02
2024-05-10 10:29:05.354 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0100 / 2634, loss 0.25020, acc 11.33%, total time [00:00], one epoch time 00:19
2024-05-10 10:29:06.009 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0200 / 2634, loss 0.07149,

## model with dynamic lmfcc with different learning rate

* dynamic lmfcc
* active_func = relu
* batch_size = 256
* hidden_layer_list = [256, 256, 256, 256]
* initial_lr = 0.0002

In [53]:
# set parameters and initiate net
use_dynamic_features = True
feature_type = 'lmfcc'
activate_func = 'relu'
batch_size = 256
hidden_layer_list = [256, 256, 256, 256]
initial_lr = 0.0002
num_epochs = 1
if feature_type == 'lmfcc':
    feat_dim = 13
elif feature_type == 'mspec':
    feat_dim = 40

if use_dynamic_features:
    input_size = feat_dim * 7
else:
    input_size = feat_dim

stateList = np.load('state_list.npy').tolist()
num_cls = len(stateList)

net = Net(input_size=input_size, num_cls=num_cls, hidden_layer_list=hidden_layer_list, activate_func=activate_func)
logger.info(net)
logger.info('number of prameters:', count_parameters(net))
net = net.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=initial_lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

# load preprocessed data
prepared_data_train = np.load('prepared_train_data.npz')
if use_dynamic_features:
    np_train_x = prepared_data_train[f'data_x_dynamic_{feature_type}']
    np_train_y = prepared_data_train['data_y']
else:
    np_train_x = prepared_data_train[f'data_x_{feature_type}']
    np_train_y = prepared_data_train['data_y']
train_x = torch.tensor(np_train_x)
train_y = F.one_hot(torch.tensor(np_train_y, dtype=torch.long), num_classes=num_cls).float()

prepared_data_val = np.load('prepared_val_data.npz')
if use_dynamic_features:
    np_val_x = prepared_data_val[f'data_x_dynamic_{feature_type}']
    np_val_y = prepared_data_val['data_y']
else:
    np_val_x = prepared_data_val[f'data_x_{feature_type}']
    np_val_y = prepared_data_val['data_y']
val_x = torch.tensor(np_val_x)
val_y = F.one_hot(torch.tensor(np_val_y, dtype=torch.long), num_classes=num_cls).float()
    
prepared_data_test = np.load('prepared_test_data.npz')
if use_dynamic_features:
    np_test_x = prepared_data_test[f'data_x_dynamic_{feature_type}']
    np_test_y = prepared_data_test['data_y']
else:
    np_test_x = prepared_data_test[f'data_x_{feature_type}']
    np_test_y = prepared_data_test['data_y']
test_x = torch.tensor(np_test_x)
test_y = F.one_hot(torch.tensor(np_test_y, dtype=torch.long), num_classes=num_cls).float()

# create the data loaders for training and validation sets
train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(val_x, val_y)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# train the network
st = time.time()
last_st = time.time()
log_interval = 100

for epoch in range(num_epochs):
    net.train()
    train_loss = 0.0
    sub_train_loss = 0.0
    epoch_acc = 0.0
    cnt = 0
    for idx, (inputs, labels) in enumerate(train_loader):

        # move data from cpu to gpu (if available)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # accumulate the training loss
        train_loss += loss.item()

        sub_train_loss += loss.item()

        if idx % log_interval == 0:
            duration = time.time() - st
            acc = calc_accuracy(labels, outputs)
            epoch_acc += (acc * batch_size)
            cnt += batch_size
            logger.info(f'epoch {epoch:03d} / {num_epochs}, step {idx:04d} / {len(train_loader)}, loss {sub_train_loss / log_interval:.5f}, acc {acc*100:.2f}%, total time [{format_time(duration)}], one epoch time {format_time(len(train_loader) / log_interval * (time.time() - last_st))}')
            sub_train_loss = 0.0
            last_st = time.time()

    # update the learning rate
    scheduler.step()

    # calculate the validation loss
    net.eval()
    val_loss = 0.0
    val_epoch_acc = 0.0
    val_cnt = 0
    with torch.no_grad():
        val_loss = 0.0
        logger.info('Validation...')
        for inputs, labels in tqdm(val_loader):
            # move data from cpu to gpu (if available)
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            acc = calc_accuracy(labels, outputs)
            val_epoch_acc += (acc * batch_size)
            val_cnt += len(inputs)

    # print the epoch loss
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    train_acc = epoch_acc/cnt*100
    val_acc = val_epoch_acc/val_cnt*100

    logger.info(f'Epoch {epoch}: train_loss={train_loss:.4f}, train_acc={train_acc:.2f}%, val_loss={val_loss:.4f}, val_acc={val_acc:.2f}%, lr={scheduler.get_last_lr()[0]}')
    writer.add_scalars('loss',{'train':train_loss,'val':val_loss},epoch)
    
# finally evaluate model on the test set here
net.eval()
test_loss = 0.0
test_epoch_acc = 0.0
test_cnt = 0
logger.info('Testing...')
with torch.no_grad():
    test_loss = 0.0
    for inputs, labels in tqdm(test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        acc = calc_accuracy(labels, outputs)
        test_epoch_acc += (acc * batch_size)
        test_cnt += len(inputs)
test_loss /= len(test_loader)
test_acc = test_epoch_acc/test_cnt*100
logger.info(f'Results: test_loss={test_loss:.4f}, test_acc={test_acc:.2f}%')

2024-05-10 10:30:06.368 | INFO     | __main__:<module>:23 - Net(
  (activate): ReLU()
  (layer1): Linear(in_features=91, out_features=256, bias=True)
  (hidden_layers): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
  )
  (header_layer): Linear(in_features=256, out_features=61, bias=True)
)
2024-05-10 10:30:06.370 | INFO     | __main__:<module>:24 - number of prameters:
2024-05-10 10:30:07.834 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0000 / 5267, loss 0.00690, acc 0.00%, total time [00:00], one epoch time 00:04
2024-05-10 10:30:08.414 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0100 / 5267, loss 0.25903, acc 15.62%, total time [00:00], one epoch time 00:30
2024-05-10 10:30:08.984 | INFO     | __main__:<module>:104 - epoch 000 / 1, step 0200 / 5267, loss 0.07235,